<a href="https://colab.research.google.com/github/IlyaZutler/MapAnalysis/blob/main/MapDistance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests

# Координаты двух точек (Долгота, Широта)
start_coords = "13.388860,52.517037"
end_coords = "13.397634,52.529407"

# URL для маршрута
url = f"http://router.project-osrm.org/route/v1/driving/{start_coords};{end_coords}?overview=false"

# Отправка запроса
response = requests.get(url)
data = response.json()

# Получение расстояния
distance = data['routes'][0]['distance'] / 1000  # в километрах
print(f"Distance: {distance} km")


Distance: 1.8868 km


In [ ]:
!pip install pandas requests geopy openpyxl
import pandas as pd
import requests
from geopy.distance import geodesic

In [ ]:
# Функция для вычисления гаверсинова расстояния
def haversine_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).kilometers

# Функция для вызова OSRM API и получения расстояния по сети дорог
def get_osrm_distance(lat1, lon1, lat2, lon2):
    url = f"http://router.project-osrm.org/route/v1/driving/{lon1},{lat1};{lon2},{lat2}?overview=false"
    response = requests.get(url)
    data = response.json()
    if 'routes' in data:
        return data['routes'][0]['distance'] / 1000  # расстояние в километрах
    else:
        return None

# Загрузка данных из Excel-файла
input_file = 'trips.xlsx'
df = pd.read_excel(input_file)

# Создание столбцов для результатов
df['Гаверсиново расстояние (км)'] = None
df['Расстояние по сети дорог (км)'] = None

# Обработка каждой строки
for index, row in df.iterrows():
    lat1, lon1 = row['Широта отправления'], row['Долгота отправления']
    lat2, lon2 = row['Широта назначения'], row['Долгота назначения']

    # Расчет гаверсинова расстояния
    haversine_dist = haversine_distance(lat1, lon1, lat2, lon2)
    df.at[index, 'Гаверсиново расстояние (км)'] = haversine_dist

    # Получение расстояния по сети дорог
    osrm_dist = get_osrm_distance(lat1, lon1, lat2, lon2)
    df.at[index, 'Расстояние по сети дорог (км)'] = osrm_dist

# Сохранение результатов в новый Excel-файл
output_file = 'trips_with_distances.xlsx'
df.to_excel(output_file, index=False)

print("Расчеты завершены и сохранены в файл:", output_file)